In [9]:
import cv2 as cv
import numpy as np
from numpy import interp
from matplotlib import pyplot as plt
import math
import json

class ImagePreprocessing():
    @staticmethod
    def imgToGrayscale():
        img0 = cv.imread('images/apple2.jpg')
        RGB_img = cv.cvtColor(img0, cv.COLOR_BGR2RGB)

        gray = RGB_img.copy()

        for y in range(len(gray)):
            for x in range(len(gray[y])):
                r = gray[y][x][0]
                g = gray[y][x][1]
                b = gray[y][x][2]
                gr = r * 0.299 + g * 0.587 + b * 0.0114 #LUMA ratios

                gray[y][x][0] = gray[y][x][1] = gray[y][x][2] = gr

        return gray
        

    @staticmethod
    def edgeDetection():
        gray = ImagePreprocessing.imgToGrayscale()
        edges = cv.Canny(gray,300,400)

        return edges
        

    @staticmethod
    def outlineVerts():
        edges = ImagePreprocessing.edgeDetection()
        arr = []

        for y in range(len(edges)):
            for x in range(len(edges[y])):
                if edges[y][x] == 255:
                    #display white values
                    #print(y,x, edges[x][y]) 
        
                    arr.append((y,x)) #row - y, col - x
        return arr

    @staticmethod
    def outlineVertsPoly():
        arr = ImagePreprocessing.outlineVerts()
        outVerts = []

        #even only
        poly = 20

        for i in range(len(arr))[::poly]:
            outVerts.append((arr[i][1], arr[i][0]))

        return outVerts

    @staticmethod
    def oposVerts():
        arr = ImagePreprocessing.outlineVerts()
        
        #even only
        poly = 20
        # #couple opposite vertices together based on the same y value
        arr2 = []
        
        for i in range(len(arr))[::poly]:
            for j in range(len(arr)):
                if (arr[i][0] == arr[j][0]):
                    if (arr[i] != arr[j]):
                        arr2.append(arr[i])
                        arr2.append(arr[j])
                        break
        
        # print('Arr2: ', arr2)
        # print('Len: ', len(arr2))
        return arr2

    
    @staticmethod
    def distCalculation():
        arr2 = ImagePreprocessing.oposVerts()
        
        #find distance between opposite vertices
        distArr = []
        
        for i in range(len(arr2)):
            n = i+1
            if (i == 0):
                dist = arr2[i][1] - arr2[n][1]
                distArr.append(abs(dist))
            else:
                if (len(arr2) % 2 != 0):
                    if (n % 2 == 0 and n != len(arr2)-1):
                        dist = arr2[n][1] - arr2[n+1][1]
                        distArr.append(abs(dist))
                else:
                    if (n % 2 == 0 and n != len(arr2)):
                        dist = arr2[n][1] - arr2[n+1][1]
                        distArr.append(abs(dist))
                        
        # print('distArr: ', distArr)
        # print(len(distArr))
        return distArr


    @staticmethod
    def innerVerts():
        from bresenham import bresenham
        arr2 = ImagePreprocessing.oposVerts()
        arr3 = []
        for i in range(len(arr2)):
            n = i+1
            if (i == 0):
                arr3.append(list(bresenham(arr2[i][1], arr2[i][0], arr2[n][1], arr2[n][0])))
            else:
                if (len(arr2) % 2 != 0):
                    if (n % 2 == 0 and n != len(arr2)-1):
                        arr3.append(list(bresenham(arr2[n][1], arr2[n][0], arr2[n+1][1], arr2[n+1][0])))
                else:
                    if (n % 2 == 0 and n != len(arr2)):
                        arr3.append(list(bresenham(arr2[n][1], arr2[n][0], arr2[n+1][1], arr2[n+1][0])))
        return arr3

    
    @staticmethod
    def addingZ():
        arr3 = ImagePreprocessing.innerVerts()
        distArr = ImagePreprocessing.distCalculation()

        #even only
        poly = 20
        
        for i in range(len(arr3)):
            firstHalf = math.ceil(len(arr3[i])/2)
            secondHalf = len(arr3[i])-firstHalf
            # print(firstHalf, secondHalf)
            # print("Len :", len(arr3[i]), firstHalf, distArr[i])
            for j in range(len(arr3[i]))[:firstHalf:]:
                # print("First half: ")
                # print(arr3[i][j])
                arr3[i][j] += (j + len(arr3[i])/distArr[i],)
                # print(arr3[i][j])
                
            for j in range(len(arr3[i]))[firstHalf::]:
                # print("Second half: ")
                temp = (j-secondHalf)*2
                # print(temp)
                arr3[i][j] += (j + len(arr3[i])/distArr[i] - temp,)    
                # print(arr3[i][j])

        verts = []

        for i in range(len(arr3)):
            for j in range(len(arr3[i])-1)[1::poly]:
                verts.append([arr3[i][j][0], arr3[i][j][1], arr3[i][j][2]])
        
        return verts


    @staticmethod
    def beforeEdgeCreation():
        verts = ImagePreprocessing.addingZ()
        
        # couple opposite vertices together based on the same y value
        arr4 = []
        temp = []
        temp1 = []
        
        #add the last singles
        for i in range(len(verts)):
            for j in range(len(verts)):
                if (verts[i][1] == verts[j][1]):
                    v = verts[i]
                    # print(v, verts[j])
                    if (verts[i] == verts[j]):
                        # print("V1: ", v)
                        temp1.append(v)
                    if (verts[i] != verts[j]):
                        # print("I: ", verts[i])
                        if (len(temp1) > 0):
                            temp1.pop()
                            # print("Temp1: ", temp1)
                            for t in range(len(temp1)):
                                arr4.append(list([temp1[t]]))
                            temp1 = []
                        if(verts[i][1] == verts[j][1]):
                            lastV = verts[i+1][1]
                            temp.append(verts[i])
                            if(len(temp) > 1 and verts[i][1] != verts[i-1][1] or verts[i][1] != lastV):
                                arr4.append(temp) 
                                temp = []
                            break

            #sort it
            for i in range(len(arr4)):
                for j in range(len(arr4[i])):
                    arr4[i].sort()
        return arr4


    @staticmethod
    def checkZWithGray():
        arr4 = ImagePreprocessing.beforeEdgeCreation()
        gray = ImagePreprocessing.imgToGrayscale()
        
        for i in range(len(arr4)):
            for j in range(len(arr4[i])):
                x = arr4[i][j][0]
                y = arr4[i][j][1]
                z = arr4[i][j][2]
                gr = gray[y][x][0]
                # print(x, y, z, gr)
        
                if (gr < 127):
                    z = interp(gray[y][x][0],[0,255],[z,z-(gray[y][x][0])/100])
                    arr4[i][j][2] = float(z)
                else:
                    z = interp(gray[y][x][0],[0,255],[z,z+(gray[y][x][0])/100])
                    arr4[i][j][2] = float(z)
                    
        return arr4

        

In [2]:
# gray = ImagePreprocessing.imgToGrayscale()

# plt.subplot(121),plt.imshow(gray)
# plt.title('Gray Image'), plt.xticks([]), plt.yticks([])

# edges = ImagePreprocessing.edgeDetection()
# plt.subplot(122),plt.imshow(edges)
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# outVerts = ImagePreprocessing.outlineVerts()
# print(outVerts)

# outVertsPoly = ImagePreprocessing.outlineVertsPoly()
# print(outVertsPoly)

# oposVerts = ImagePreprocessing.oposVerts()
# print(oposVerts)

# distCalcArr = ImagePreprocessing.distCalculation()
# print(distCalcArr)

# innerVertsArr = ImagePreprocessing.innerVerts()
# print(innerVertsArr)

# zCoords = ImagePreprocessing.addingZ()
# print(zCoords)

# beforeEdges = ImagePreprocessing.beforeEdgeCreation()
# print(beforeEdges)

# grayZ = ImagePreprocessing.checkZWithGray()
# print(grayZ)

In [10]:
import numpy as np
import math
import random

class ObjectCreation():
    def __init__(self):
        self.vertsToDicts = ObjectCreation.vertsToDicts()
        self.fitnessArr = ObjectCreation.fitnessDict(self.vertsToDicts)
        self.objF = ObjectCreation.objFitness(self.fitnessArr)
        self.blenderVerts = ObjectCreation.vertsForBlender(self.fitnessArr)
        self.blenderEdges = ObjectCreation.edgesForBlender(self.vertsToDicts)
# print(blenderEdges)
    
    @staticmethod
    #gene creation
    def vertsToDicts():
        arr4 = ImagePreprocessing.checkZWithGray()
        
        arr5 = arr4
        count = -1
        
        for i in range(len(arr4)):
            for j in range(len(arr4[i])):
                count +=1
                arr5[i][j] = {"id": count, "data": arr4[i][j], "fitness_val": 0}

        #randomize internal vectors' x and z values
        for i in range(len(arr5)):
            for j in range(len(arr5[i])):
                if(len(arr5[i]) > 2):
                    if (j != 0 and j != len(arr5[i])-1):
                        # randX = random.randrange(arr5[i][j-1]['data'][0], arr5[i][j+1]['data'][0])
                        randX = random.triangular(arr5[i][j-1]['data'][0], arr5[i][j+1]['data'][0])
                        randZ = random.triangular(arr5[i][j-1]['data'][2], arr5[i][j+1]['data'][2])
                        arr5[i][j]['data'] = [randX, arr5[i][j]['data'][1], randZ]
                        
        return arr5

    @staticmethod
    def fitnessDict(vToDicts):
        # arr5 = ObjectCreation.vertsToDicts()
        arr5 = vToDicts
        fitArr = arr5
        
        for i in range(len(arr5)):
            for j in range(len(arr5[i])):
                randomForce = round(random.triangular(0, 100), 2)
                # print(randomForce, fitArr[i][j]['data'])

                qx = round(randomForce/fitArr[i][j]['data'][0], 2)
                qy = round(randomForce/fitArr[i][j]['data'][1], 2)
                qz = round(randomForce/fitArr[i][j]['data'][2], 2)

                #the lesser the better - more resistance
                q = 0
                qMain = (qx + qy + qz)
                if (qMain != 0):
                    q = round(1/qMain, 3)
                else:
                    q = round(1/1, 3)
                # print(q)
                fitArr[i][j]['fitness_val'] = q

        return fitArr
    
    @staticmethod
    def objFitness(fitArrPassed):
        # fitArr = ObjectCreation.fitnessDict()
        fitArr = fitArrPassed

        objF = 0

        for i in range(len(fitArr)):
            for j in range(len(fitArr[i])):
                # print(fitArr[i][j]['data'], fitArr[i][j]['fitness_val'])
                objF += fitArr[i][j]['fitness_val']

        return round(objF, 3)
        
    
    @staticmethod
    def vertsForBlender(fitDict):
        # arr5 = ObjectCreation.fitnessDict()
        arr5 = fitDict
        
        #blender ready verts arr
        verts = []
        
        for i in range(len(arr5)):
            for j in range(len(arr5[i])):
                verts.append(arr5[i][j]['data'])

        return verts


    @staticmethod
    def edgesForBlender(vToDicts):
        # arr5 = ObjectCreation.vertsToDicts()
        arr5 = vToDicts
        
        #add to edges array in the form of [[0, 1], [1, 2]]
        edges = []
        
        for i in range(len(arr5)):
            for j in range(len(arr5[i])):
                if(len(arr5[i]) < 2):  
                    edges.append([arr5[i][j]['id'], arr5[i+1][j]['id']])
                else:
                    #when next array is larger or smaller than the current array
                    if (i != len(arr5)-1):
                        if (len(arr5[i]) != len(arr5[i+1])):
                            #when j is not the last index
                            if (j != len(arr5[i])-1):
                                edges.append([arr5[i][j]['id'], arr5[i][j+1]['id']])
                
                                if (i != len(arr5)-1):  
                                    tempDist = []
                
                                    if(j == 0):
                                        count = 0
                                    else:
                                        count = ind
                
                                    for c in range(len(arr5[i+1]))[count::]:
                                        x = pow(abs(arr5[i][j]['data'][0] - arr5[i+1][c]['data'][0]), 2)
                                        y = pow(abs(arr5[i][j]['data'][1] - arr5[i+1][c]['data'][1]), 2)
                                        dist = round(math.sqrt(x + y), 2)
                
                                        edges.append([arr5[i][j]['id'], arr5[i+1][c]['id']])
                                        tempDist.append(dist)
                                        # print(arr5[i][j]['id'], arr5[i+1][c]['id'], tempDist)
                                        
                                        if (len(tempDist) > 0 and dist > tempDist[0]):
                                            edges.pop()
                                            ind = c-1
                                            break
                            # when j is the last index
                            else:
                                if(i != len(arr5)-1):
                                    for c in range(len(arr5[i+1]))[ind::]:
                                      edges.append([arr5[i][j]['id'], arr5[i+1][c]['id']])
                        #when arrays are equal
                        else:
                            if (j != len(arr5[i])-1):
                                edges.append([arr5[i][j]['id'], arr5[i][j+1]['id']])
                                if (i != len(arr5)-1):
                                    edges.append([arr5[i][j]['id'], arr5[i+1][j]['id']])
                            else:
                                if(i != len(arr5)-1):
                                    edges.append([arr5[i][j]['id'], arr5[i+1][len(arr5[i+1])-1]['id']])
        
        
        return edges

    @staticmethod
    def toJson(self, id):
        import json

        crDict = {
            'objDict': [],
            'fitNum': 0,
            'objEdges': []
        
        }
        
        # crDict['objDict'] = ObjectCreation.fitnessDict()
        # crDict['fitNum'] = ObjectCreation.objFitness()
        # crDict['objEdges'] = ObjectCreation.edgesForBlender()

        crDict['objDict'] = self.fitnessArr
        crDict['fitNum'] = self.objF
        crDict['objEdges'] = self.blenderEdges

        with open('creatures_json/objData_' + str(id) + '.txt', 'w') as json_file:
            json.dump(crDict, json_file)

        return json_file

    
    @staticmethod
    def createChildDNA(self, child, newGen, pool_gen, gen):
        import json
        
        self.vertsToDicts = child
        self.fitnessArr = ObjectCreation.fitnessDict(self.vertsToDicts)
        self.objF = ObjectCreation.objFitness(self.fitnessArr)
        self.blenderVerts = ObjectCreation.vertsForBlender(self.fitnessArr)
        self.blenderEdges = ObjectCreation.edgesForBlender(self.vertsToDicts)

        chDict = {
        'objDict': [],
        'fitNum': 0,
        'objVerts': [],
        'objEdges': []
        }

        chDict['objDict'] = self.fitnessArr
        chDict['fitNum'] = self.objF
        chDict['objVerts'] = self.blenderVerts
        chDict['objEdges'] = self.blenderEdges

        if (newGen == False):
            with open('creatures_json/childDict_poolGen_' + str(pool_gen) + '_gen_' + str(gen) + '.txt', 'w') as json_file:
                json.dump(chDict, json_file)
        else:
            with open('creatures_json/newChildDict_poolGen_' + str(pool_gen) + '_gen_' + str(gen) + '.txt', 'w') as json_file:
                json.dump(chDict, json_file)

        return chDict


In [11]:
vertsToDicts = ObjectCreation.vertsToDicts()
# print(vertsToDicts)

fitnessArr = ObjectCreation.fitnessDict(vertsToDicts)
# print(fitnessArr)

# objF = ObjectCreation.objFitness(fitnessArr)
# # print(objF)

blenderVerts = ObjectCreation.vertsForBlender(fitnessArr)
print(blenderVerts)

blenderEdges = ObjectCreation.edgesForBlender(vertsToDicts)
print(blenderEdges)

# json = ObjectCreation.toJson(ObjectCreation(), 1)
# json

# with open('creatures_json/childData.txt', "r") as file:
#     childData = json.load(file)

# smth = ObjectCreation.createChildDNA(ObjectCreation(), childData, 1)
# print(smth)


[[154, 137, 3.4656470588235297], [154, 147, 2.933294117647059], [131, 155, 2.3132941176470587], [127, 156, 2.034078431372549], [129, 156, 5.169433155080214], [149, 156, 21.796217468805704], [169, 156, 1.6688057040998214], [121, 161, 7.506327985739751], [141, 161, 27.328445632798573], [161, 161, 22.851073083778967], [181, 161, 2.8625632798573974], [116, 164, 5.4834723707664885], [136, 164, 23.27923707664884], [156, 164, 1.8953155080213901], [113, 171, 9.65327659574468], [133, 171, 23.334570713391738], [153, 171, 3.050453066332916], [119, 174, 17.53244788441693], [139, 174, 37.38185964912281], [159, 174, 43.295271413828694], [179, 174, 23.03970278637771], [199, 174, 1.578173374613003], [102, 181, 3.103490196078431], [122, 181, 23.562784313725487], [142, 181, 43.38113725490196], [162, 181, 43.32309803921569], [182, 181, 23.10349019607844], [202, 181, 3.1699607843137305], [99, 189, 3.3224684295736644], [119, 189, 23.409997841338374], [139, 189, 43.36588019427955], [159, 189, 51.27976254722

In [5]:
#import objectCreation
class Population():
    def __init__(self, pop_size):
        self.creatures = [ObjectCreation() for i in range(pop_size)]

    @staticmethod
    def get_fitness_map(fits):
        fitmap = []
        total = 0
        for f in fits:
            total += f
            fitmap.append(round(total, 3))
        return fitmap

    @staticmethod
    def select_parent(fitmap):
        r = round(random.triangular(0, fitmap[-1]), 2)
        print('r: ', r)
        for i in range(len(fitmap)):
            if r <= fitmap[i]:
                return i

In [6]:
import json
import random

class Evolution():
    def __init__(self, pool_size, gen_size):
        self.pool_size = pool_size #has to be more than 1
        self.gen_size = gen_size
        # self.pool = Evolution.popCreation(self.pool_size, self.gen_size)
        # self.parentID = Evolution.popSelection(self.pool_size, self.gen_size, pool)
        
        # self.parentsArr = Evolution.newGenCreation(self.pool_size, self.gen_size)
        # self.g3 = Evolution.crossover(self.pool_size, self.gen_size, self.parentsArr)
        self.newGen = Evolution.generationCreation(self, self.pool_size, self.gen_size)
    
    @staticmethod
    def popCreation(p, g):
        pop = Population(p)
        
        for i in range(len(pop.creatures)):
            pop.creatures[i].toJson(pop.creatures[i], i)

        pool = pop.creatures

        return pool
            

    @staticmethod
    def popSelection(p, g, pool):
        fits = []
        
        for i in range(len(pool)):
            with open('creatures_json/objData_' + str(i) + '.txt', "r") as file:
                data = json.load(file)
                fitN = data['fitNum']
                fits.append(fitN)
          
        print('Fits: ', fits)
              
        #print fitness map
        fitmap = Population(len(pool)).get_fitness_map(fits)
        print('Fitmap: ', fitmap)

        #print selected parent
        parentID = Population(len(pool)).select_parent(fitmap)
        print('Parent id: ', parentID)

        return parentID

    @staticmethod
    def newGenCreation(self, p, g, end_of_gen, newParent, new_parents_arr):
        if (end_of_gen == False):
            parents = []
            #2 parents
            for i in range(2):
                parent = Evolution.popSelection(self.pool_size, self.gen_size, Evolution.popCreation(self.pool_size, self.gen_size))
                print('newGenCreation: ', parent)
                parents.append(parent)
                
                with open('creatures_json/objData_' + str(parent) + '.txt', "r") as file:
                    data = json.load(file)
        
                with open('creatures_json/parentData_pool_' + str(i) + '_parent_'+ str(parent) + '_poolGen_' + str(p) + '_gen_' + str(g) + '.txt', 'w') as json_file:
                    json.dump(data, json_file)
            return parents
        else:
            new_parent = newParent
            print('newParentsCreation: ', new_parent)

            if(len(new_parents_arr) < 2):
                with open('creatures_json/childDict_poolGen_' + str(len(new_parents_arr)-1) + '_gen_' + str(g) + '.txt', "r") as file:
                    data = json.load(file)
                with open('creatures_json/newParentData_pool_'+ str(0) + '.txt', 'w') as json_file:
                    json.dump(data, json_file)
            else:
                with open('creatures_json/childDict_poolGen_' + str(len(new_parents_arr)-1) + '_gen_' + str(g) + '.txt', "r") as file:
                    data = json.load(file)
                with open('creatures_json/newParentData_pool_'+ str(1) + '.txt', 'w') as json_file:
                    json.dump(data, json_file)

            # print(new_parent)

            return new_parent

                
    
    @staticmethod
    def crossover(p, g, parents, newGen):
        # ps = Evolution(self.pool_size, self.gen_size).newGenCreation(Evolution(self.pool_size, self.gen_size))
        # print(ps)
        pNum = p
        gNum = g

        p1 = {}
        p2 = {}

        if (newGen == False):
            with open('creatures_json/parentData_pool_0_parent_' + str(parents[0]) + '_poolGen_' + str(p) + '_gen_' + str(g) + '.txt', "r") as file:
                    p1 = json.load(file)
    
            with open('creatures_json/parentData_pool_1_parent_' + str(parents[1]) + '_poolGen_' + str(p) + '_gen_' + str(g) + '.txt', "r") as file:
                    p2 = json.load(file)
        else: 
            print('New gen starts')
             
            with open('creatures_json/newParentData_pool_0.txt', "r") as file:
                    p1 = json.load(file)
    
            with open('creatures_json/newParentData_pool_1.txt', "r") as file:
                    p2 = json.load(file)

        # print('Parent1')
        # print(p1)
        # print('Parent2')
        # print(p2)

        g3 = p1['objDict']
        # print(g3)

        for i in range(len(p1['objDict'])):
            for j in range(len(p1['objDict'][i])):
                # print(p1['objDict'][i][j], p2['objDict'][i][j])
                
                p = random.randint(50, 100)
                # print(p)
                
                if (p1['objDict'][i][j]['fitness_val'] > p2['objDict'][i][j]['fitness_val']):
                    fittest = p1['objDict'][i][j]
                    lessFittest = p2['objDict'][i][j]
                else:
                    fittest = p2['objDict'][i][j]
                    lessFittest = p1['objDict'][i][j]
                    
                # print("Fittest: ", fittest)
                # print("Less fittest: ", lessFittest)
        
                x = abs(p1['objDict'][i][j]['data'][0] - p2['objDict'][i][j]['data'][0])
                # print('X: ', x)
                
                distX = round(x * p/100)
                # print('DistX: ', distX)
        
                #check x vals
                if(fittest['data'][0] > lessFittest['data'][0]):
                    fittestXLarger = True
                else:
                    fittestXLarger = False
                
                # print(fittestXLarger)
        
                #toward p with lower f.val
                if (fittestXLarger):
                    g3[i][j]['data'][0] = fittest['data'][0] - distX 
                else:
                    g3[i][j]['data'][0] = fittest['data'][0] + distX
        
                z = abs(p1['objDict'][i][j]['data'][2] - p2['objDict'][i][j]['data'][2])
                # print('Z: ', z)
                
                distZ = round(z * p/100)
                # print('DistZ: ', distZ)
        
                #check z vals
                if(fittest['data'][2] > lessFittest['data'][2]):
                    fittestZLarger = True
                else:
                    fittestZLarger = False
                
                # print(fittestZLarger)
        
                #toward p with higher f.val
                if (fittestZLarger):
                    g3[i][j]['data'][2] = lessFittest['data'][2] + distZ 
                else:
                    g3[i][j]['data'][2] = lessFittest['data'][2] - distZ

                g3[i][j]['fitness_val'] = 0
                    
        # print('G3: ', g3)

        #mutation
        for i in range(len(g3)):
            for j in range(len(g3[i])):
                if(len(g3[i]) > 2):
                    if (j != 0 and j != len(g3[i])-1):
                        # print('G3: ', 3[i][j])
                        r = random.randint(0, 10)
                        # print(r)
                        
                        g3[i][j]['data'][2] += r

        # print('G3M: ', g3)
        
        if (newGen == False):
            with open('creatures_json/childData.txt', 'w') as json_file:
                json.dump(g3, json_file)
    
            with open('creatures_json/childData.txt', "r") as file:
                childData = json.load(file)
                
            g3Json = ObjectCreation.createChildDNA(ObjectCreation(), childData, newGen, pNum, gNum)
        else:
            with open('creatures_json/newChildData.txt', 'w') as json_file:
                json.dump(g3, json_file)
    
            with open('creatures_json/newChildData.txt', "r") as file:
                childData = json.load(file)

            g3Json = ObjectCreation.createChildDNA(ObjectCreation(), childData, newGen, pNum, gNum)

        # print('P: ', pNum, str(pNum))
        
        return g3Json

    @staticmethod
    def generationCreation(self, p, g):
        # parents = Evolution.newGenCreation(p, g)
        new_gen = []
        new_parents = []
        end_of_gen = False
        newGen = False
        
        for j in range(g):
            print('J: ', j)
            for i in range(p):
                print('I: ', i)
                print('J: ', j)
                
                self.parentsArr = Evolution.newGenCreation(self, i, j, end_of_gen, None, None)
                self.g3 = Evolution.crossover(i, j, self.parentsArr, newGen)
                
                new_gen.append(self.g3)
                
            if (len(new_gen) == p):
                print('end of gen')
                end_of_gen = True
                
                fits = []
                for i in range(len(new_gen)): 
                    with open('creatures_json/childDict_poolGen_' + str(i) + '_gen_' + str(j) + '.txt', "r") as file:
                        data = json.load(file)
                        fitN = data['fitNum']
                        fits.append(fitN)
                  
                print('Fits: ', fits)
                      
                #print fitness map
                fitmap = Population(len(new_gen)).get_fitness_map(fits)
                print('Fitmap: ', fitmap)
        
                #print selected parent
                parentID = Population(len(new_gen)).select_parent(fitmap)
                print('Parent id: ', parentID)

                new_parents.append(parentID)
                print('New parents arr: ', new_parents)
                parentsArr = Evolution.newGenCreation(self, i, j, end_of_gen, parentID, new_parents)
                
                new_gen = []
                end_of_gen = False

                if(len(new_parents) > 2):
                    new_parents = []
                    print('New parents arr cleared: ', new_parents)
                   
                if (len(new_parents) > 1):
                    newGen = True
                    self.g3 = Evolution.crossover(p, g, new_parents, newGen)
                    new_gen.append(self.g3)
                    newGen = False

        
        with open('creatures_json/renderObj.txt', 'w') as json_file:
            json.dump(new_gen[0], json_file)
                    
        return new_gen


In [7]:
# pool = Evolution(2,1).popCreation(Evolution(2,1))
# print('Pop Creation: ', pool)

# popSelect = Evolution(2,1).popSelection(Evolution(2,1))
# print('ParentID: ', popSelect) 

# parents = Evolution(1,1).newGenCreation(Evolution(1,1))
# print("Parents: ", parents)

# cros = Evolution(2,1).crossover(Evolution(2, 1))
# cros

Evolution(2,2)


# pop_num = 2
# gen_num = 1
# new_gen = Evolution(pop_num, gen_num).generationCreation(pop_num, gen_num)
# print(type(new_gen), len(new_gen))

J:  0
I:  0
J:  0
Fits:  [30.85, 22.334]
Fitmap:  [30.85, 53.184]
r:  45.62
Parent id:  1
newGenCreation:  1
Fits:  [25.415, 24.174]
Fitmap:  [25.415, 49.589]
r:  36.62
Parent id:  1
newGenCreation:  1
I:  1
J:  0
Fits:  [24.194, 23.838]
Fitmap:  [24.194, 48.032]
r:  23.56
Parent id:  0
newGenCreation:  0
Fits:  [23.312, 30.631]
Fitmap:  [23.312, 53.943]
r:  9.96
Parent id:  0
newGenCreation:  0
end of gen
Fits:  [26.218, 27.276]
Fitmap:  [26.218, 53.494]
r:  32.76
Parent id:  1
New parents arr:  [1]
newParentsCreation:  1
J:  1
I:  0
J:  1
Fits:  [36.09, 29.995]
Fitmap:  [36.09, 66.085]
r:  23.15
Parent id:  0
newGenCreation:  0
Fits:  [26.864, 32.829]
Fitmap:  [26.864, 59.693]
r:  15.82
Parent id:  0
newGenCreation:  0
I:  1
J:  1
Fits:  [24.849, 21.375]
Fitmap:  [24.849, 46.224]
r:  27.67
Parent id:  1
newGenCreation:  1
Fits:  [26.754, 35.089]
Fitmap:  [26.754, 61.843]
r:  33.75
Parent id:  1
newGenCreation:  1
end of gen
Fits:  [31.216, 27.816]
Fitmap:  [31.216, 59.032]
r:  39.99


In [34]:
# another class to render the obj
# RenderFinalObj class

import json
with open('creatures_json/renderObj.txt', "r") as file:
    data = json.load(file)

# print(data)

verts = data['objVerts']
edges = data['objEdges']

print(edges)

[[0, 1], [1, 2], [2, 3], [3, 4], [3, 7], [4, 5], [4, 8], [5, 6], [5, 9], [6, 10], [7, 8], [7, 11], [8, 9], [8, 11], [8, 12], [8, 13], [9, 10], [9, 11], [9, 12], [9, 13], [10, 11], [10, 12], [10, 13], [11, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, 15], [14, 17], [15, 16], [15, 17], [15, 18], [16, 18], [16, 19], [16, 20], [16, 21], [17, 18], [17, 22], [17, 23], [18, 19], [18, 23], [19, 20], [19, 23], [19, 24], [19, 25], [19, 26], [19, 27], [20, 21], [20, 23], [20, 24], [20, 25], [20, 26], [20, 27], [21, 23], [21, 24], [21, 25], [21, 26], [21, 27], [22, 23], [22, 28], [23, 24], [23, 29], [24, 25], [24, 30], [25, 26], [25, 31], [26, 27], [26, 32], [27, 33], [28, 29], [28, 34], [29, 30], [29, 35], [30, 31], [30, 36], [31, 32], [31, 37], [32, 33], [32, 38], [33, 39], [34, 35], [34, 40], [35, 36], [35, 41], [36, 37], [36, 42], [37, 38], [37, 43], [38, 39], [38, 44], [39, 45], [40, 41], [40, 46], [41, 42], [41, 47], [42, 43], [42, 48], [43, 44], [43, 49], [44, 45], [44, 50], [45, 51], 

In [4]:
import json

class RenderFinalObj():
    # def __init__(self):
    #     self.data = RenderFinalObj.readFile()
    #     self.verts = RenderFinalObj.objVertice(self.data)
    #     self.edges = RenderFinalObj.objEdges(self.data)

    @staticmethod
    def readFile():
        with open('creatures_json/renderObj.txt', "r") as file:
            data = json.load(file)

        return data
    
    @staticmethod
    def objVertices():
        data = RenderFinalObj.readFile()
        return data['objVerts']
    
    @staticmethod
    def objEdges():
        data = RenderFinalObj.readFile()
        return data['objEdges']

In [5]:
#file = RenderFinalObj().readFile()
# print(file)

# verts = RenderFinalObj().objVertices()
# print(verts)

# edges = RenderFinalObj().objEdges()
# print(edges)

[[0, 1], [1, 2], [2, 3], [3, 4], [3, 7], [4, 5], [4, 8], [5, 6], [5, 9], [6, 10], [7, 8], [7, 11], [8, 9], [8, 11], [8, 12], [8, 13], [9, 10], [9, 11], [9, 12], [9, 13], [10, 11], [10, 12], [10, 13], [11, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, 15], [14, 17], [15, 16], [15, 17], [15, 18], [16, 18], [16, 19], [16, 20], [16, 21], [17, 18], [17, 22], [17, 23], [18, 19], [18, 23], [19, 20], [19, 23], [19, 24], [19, 25], [19, 26], [19, 27], [20, 21], [20, 23], [20, 24], [20, 25], [20, 26], [20, 27], [21, 23], [21, 24], [21, 25], [21, 26], [21, 27], [22, 23], [22, 28], [23, 24], [23, 29], [24, 25], [24, 30], [25, 26], [25, 31], [26, 27], [26, 32], [27, 33], [28, 29], [28, 34], [29, 30], [29, 35], [30, 31], [30, 36], [31, 32], [31, 37], [32, 33], [32, 38], [33, 39], [34, 35], [34, 40], [35, 36], [35, 41], [36, 37], [36, 42], [37, 38], [37, 43], [38, 39], [38, 44], [39, 45], [40, 41], [40, 46], [41, 42], [41, 47], [42, 43], [42, 48], [43, 44], [43, 49], [44, 45], [44, 50], [45, 51], 

In [ ]:
p1 = {'objDict': [[{'id': 0, 'data': [154, 137, 3.4656470588235297], 'fitness_val': 0.055}], [{'id': 1, 'data': [154, 147, 2.933294117647059], 'fitness_val': 0.057}], [{'id': 2, 'data': [131, 155, 2.3132941176470587], 'fitness_val': 0.043}], [{'id': 3, 'data': [127, 156, 2.034078431372549], 'fitness_val': 0.032}, {'id': 4, 'data': [134, 156, 14.360384539937257], 'fitness_val': 0.225}, {'id': 5, 'data': [137, 156, 12.396760736653265], 'fitness_val': 0.253}, {'id': 6, 'data': [169, 156, 1.6688057040998214], 'fitness_val': 0.277}], [{'id': 7, 'data': [121, 161, 7.506327985739751], 'fitness_val': 0.089}, {'id': 8, 'data': [125, 161, 16.716201818446333], 'fitness_val': 0.369}, {'id': 9, 'data': [156, 161, 13.190383440835049], 'fitness_val': 0.134}, {'id': 10, 'data': [181, 161, 2.8625632798573974], 'fitness_val': 0.062}], [{'id': 11, 'data': [116, 164, 5.4834723707664885], 'fitness_val': 0.083}, {'id': 12, 'data': [136, 164, 2.3376542385369588], 'fitness_val': 0.034}, {'id': 13, 'data': [156, 164, 1.8953155080213901], 'fitness_val': 0.023}], [{'id': 14, 'data': [113, 171, 9.65327659574468], 'fitness_val': 0.282}, {'id': 15, 'data': [138, 171, 4.324017956638113], 'fitness_val': 0.196}, {'id': 16, 'data': [153, 171, 3.050453066332916], 'fitness_val': 0.059}], [{'id': 17, 'data': [119, 174, 17.53244788441693], 'fitness_val': 0.781}, {'id': 18, 'data': [144, 174, 34.69252013253842], 'fitness_val': 0.314}, {'id': 19, 'data': [153, 174, 30.5691529507397], 'fitness_val': 0.366}, {'id': 20, 'data': [164, 174, 4.657326909497652], 'fitness_val': 0.069}, {'id': 21, 'data': [199, 174, 1.578173374613003], 'fitness_val': 0.069}], [{'id': 22, 'data': [102, 181, 3.103490196078431], 'fitness_val': 0.243}, {'id': 23, 'data': [126, 181, 32.99054486720438], 'fitness_val': 0.546}, {'id': 24, 'data': [137, 181, 39.72827554921964], 'fitness_val': 2.083}, {'id': 25, 'data': [172, 181, 26.911583456606273], 'fitness_val': 0.429}, {'id': 26, 'data': [200, 181, 17.2435106811142], 'fitness_val': 0.345}, {'id': 27, 'data': [202, 181, 3.1699607843137305], 'fitness_val': 0.061}], [{'id': 28, 'data': [99, 189, 3.3224684295736644], 'fitness_val': 0.058}, {'id': 29, 'data': [116, 189, 14.092674588181325], 'fitness_val': 0.24}, {'id': 30, 'data': [127, 189, 19.93044282460988], 'fitness_val': 0.205}, {'id': 31, 'data': [146, 189, 26.012005552366823], 'fitness_val': 0.22}, {'id': 32, 'data': [155, 189, 15.56029529236443], 'fitness_val': 0.49}, {'id': 33, 'data': [199, 189, 11.223919409965816], 'fitness_val': 0.459}], [{'id': 34, 'data': [97, 199, 3.5001050420168065], 'fitness_val': 0.07}, {'id': 35, 'data': [113, 199, 6.2776453864056], 'fitness_val': 0.174}, {'id': 36, 'data': [117, 199, 24.29512182254947], 'fitness_val': 0.518}, {'id': 37, 'data': [176, 199, 29.540568727192372], 'fitness_val': 0.382}, {'id': 38, 'data': [189, 199, 18.04628389586627], 'fitness_val': 0.279}, {'id': 39, 'data': [197, 199, 13.155634453781511], 'fitness_val': 0.297}], [{'id': 40, 'data': [97, 209, 3.6409285714285713], 'fitness_val': 0.043}, {'id': 41, 'data': [115, 209, 11.950083155498742], 'fitness_val': 0.379}, {'id': 42, 'data': [129, 209, 24.45302316410571], 'fitness_val': 0.671}, {'id': 43, 'data': [151, 209, 32.37647005208676], 'fitness_val': 0.769}, {'id': 44, 'data': [157, 209, 30.74548375649654], 'fitness_val': 0.275}, {'id': 45, 'data': [197, 209, 13.959085434173668], 'fitness_val': 0.415}], [{'id': 46, 'data': [98, 219, 3.2516007130124773], 'fitness_val': 0.057}, {'id': 47, 'data': [134, 219, 27.510756467708685], 'fitness_val': 0.546}, {'id': 48, 'data': [134, 219, 41.823202592017786], 'fitness_val': 1.149}, {'id': 49, 'data': [160, 219, 40.77595017229935], 'fitness_val': 0.943}, {'id': 50, 'data': [196, 219, 27.082008975347158], 'fitness_val': 1.02}, {'id': 51, 'data': [198, 219, 11.102777183600718], 'fitness_val': 0.163}], [{'id': 52, 'data': [102, 228, 5.381042071197407], 'fitness_val': 0.074}, {'id': 53, 'data': [102, 228, 20.488216357753817], 'fitness_val': 0.763}, {'id': 54, 'data': [134, 228, 28.20382791797211], 'fitness_val': 0.559}, {'id': 55, 'data': [181, 228, 24.56817959836044], 'fitness_val': 0.388}, {'id': 56, 'data': [197, 228, 17.399845343775574], 'fitness_val': 0.208}, {'id': 57, 'data': [202, 228, 2.7783361888444698], 'fitness_val': 0.043}], [{'id': 58, 'data': [119, 235, 17.441044117647063], 'fitness_val': 0.243}, {'id': 59, 'data': [139, 235, 36.12610254945736], 'fitness_val': 0.709}, {'id': 60, 'data': [162, 235, 30.327533787586262], 'fitness_val': 0.538}, {'id': 61, 'data': [181, 235, 21.082070007938466], 'fitness_val': 0.299}, {'id': 62, 'data': [199, 235, 1.4457107843137258], 'fitness_val': 0.062}], [{'id': 63, 'data': [116, 241, 11.487032679738556], 'fitness_val': 0.322}, {'id': 64, 'data': [116, 241, 20.50027241044232], 'fitness_val': 0.263}, {'id': 65, 'data': [155, 241, 21.852346392417], 'fitness_val': 0.275}, {'id': 66, 'data': [166, 241, 8.020672020769352], 'fitness_val': 0.216}, {'id': 67, 'data': [196, 241, 1.700483660130719], 'fitness_val': 0.039}], [{'id': 68, 'data': [112, 247, 2.6447049258727913], 'fitness_val': 0.088}, {'id': 69, 'data': [117, 247, 17.033507466968228], 'fitness_val': 0.552}, {'id': 70, 'data': [136, 247, 22.477725107595408], 'fitness_val': 0.862}, {'id': 71, 'data': [177, 247, 13.327093765596384], 'fitness_val': 0.222}, {'id': 72, 'data': [192, 247, 1.6802735533237685], 'fitness_val': 0.024}], [{'id': 73, 'data': [124, 258, 5.506801470588235], 'fitness_val': 0.095}, {'id': 74, 'data': [126, 258, 13.571358631936405], 'fitness_val': 0.149}, {'id': 75, 'data': [134, 258, 8.334924994287748], 'fitness_val': 0.136}, {'id': 76, 'data': [184, 258, 1.683703431372549], 'fitness_val': 0.037}], [{'id': 77, 'data': [139, 263, 15.178675381263618], 'fitness_val': 0.248}, {'id': 78, 'data': [172, 263, 8.394256442305473], 'fitness_val': 0.124}, {'id': 79, 'data': [179, 263, 1.6793420479302832], 'fitness_val': 0.029}], [{'id': 80, 'data': [133, 267, 4.675844049247608], 'fitness_val': 0.056}, {'id': 81, 'data': [167, 267, 4.218562918978177], 'fitness_val': 0.272}, {'id': 82, 'data': [173, 267, 1.8026675786593707], 'fitness_val': 0.039}]], 'fitNum': 27.949, 'objEdges': [[0, 1], [1, 2], [2, 3], [3, 4], [3, 7], [4, 5], [4, 8], [5, 6], [5, 9], [6, 10], [7, 8], [7, 11], [7, 12], [8, 9], [8, 12], [8, 13], [9, 10], [9, 12], [9, 13], [10, 12], [10, 13], [11, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, 15], [14, 17], [15, 16], [15, 17], [15, 18], [16, 18], [16, 19], [16, 20], [16, 21], [17, 18], [17, 22], [17, 23], [18, 19], [18, 23], [18, 24], [18, 25], [19, 20], [19, 25], [20, 21], [20, 25], [20, 26], [20, 27], [21, 25], [21, 26], [21, 27], [22, 23], [22, 28], [23, 24], [23, 29], [24, 25], [24, 30], [25, 26], [25, 31], [26, 27], [26, 32], [27, 33], [28, 29], [28, 34], [29, 30], [29, 35], [30, 31], [30, 36], [31, 32], [31, 37], [32, 33], [32, 38], [33, 39], [34, 35], [34, 40], [35, 36], [35, 41], [36, 37], [36, 42], [37, 38], [37, 43], [38, 39], [38, 44], [39, 45], [40, 41], [40, 46], [41, 42], [41, 47], [42, 43], [42, 48], [43, 44], [43, 49], [44, 45], [44, 50], [45, 51], [46, 47], [46, 52], [47, 48], [47, 53], [48, 49], [48, 54], [49, 50], [49, 55], [50, 51], [50, 56], [51, 57], [52, 53], [52, 58], [53, 54], [53, 58], [54, 55], [54, 58], [54, 59], [54, 60], [54, 61], [55, 56], [55, 61], [56, 57], [56, 61], [57, 61], [57, 62], [58, 59], [58, 63], [59, 60], [59, 64], [60, 61], [60, 65], [61, 62], [61, 66], [62, 67], [63, 64], [63, 68], [64, 65], [64, 69], [65, 66], [65, 70], [66, 67], [66, 71], [67, 72], [68, 69], [68, 73], [69, 70], [69, 73], [70, 71], [70, 73], [70, 74], [70, 75], [70, 76], [71, 72], [71, 73], [71, 74], [71, 75], [71, 76], [72, 73], [72, 74], [72, 75], [72, 76], [73, 74], [73, 77], [74, 75], [74, 77], [74, 78], [74, 79], [75, 76], [75, 77], [75, 78], [75, 79], [76, 77], [76, 78], [76, 79], [77, 78], [77, 80], [78, 79], [78, 81], [79, 82]]}
p2 = {'objDict': [[{'id': 0, 'data': [154, 137, 3.4656470588235297], 'fitness_val': 0.081}], [{'id': 1, 'data': [154, 147, 2.933294117647059], 'fitness_val': 0.04}], [{'id': 2, 'data': [131, 155, 2.3132941176470587], 'fitness_val': 0.04}], [{'id': 3, 'data': [127, 156, 2.034078431372549], 'fitness_val': 0.021}, {'id': 4, 'data': [132, 156, 17.675402423839277], 'fitness_val': 0.202}, {'id': 5, 'data': [154, 156, 12.451082095553154], 'fitness_val': 0.33}, {'id': 6, 'data': [169, 156, 1.6688057040998214], 'fitness_val': 0.018}], [{'id': 7, 'data': [121, 161, 7.506327985739751], 'fitness_val': 0.075}, {'id': 8, 'data': [123, 161, 15.088154297588254], 'fitness_val': 0.394}, {'id': 9, 'data': [144, 161, 12.201212163220644], 'fitness_val': 0.22}, {'id': 10, 'data': [181, 161, 2.8625632798573974], 'fitness_val': 0.056}], [{'id': 11, 'data': [116, 164, 5.4834723707664885], 'fitness_val': 0.076}, {'id': 12, 'data': [148, 164, 3.384027765344472], 'fitness_val': 0.055}, {'id': 13, 'data': [156, 164, 1.8953155080213901], 'fitness_val': 0.027}], [{'id': 14, 'data': [113, 171, 9.65327659574468], 'fitness_val': 0.376}, {'id': 15, 'data': [126, 171, 6.656670515185469], 'fitness_val': 0.095}, {'id': 16, 'data': [153, 171, 3.050453066332916], 'fitness_val': 0.093}], [{'id': 17, 'data': [119, 174, 17.53244788441693], 'fitness_val': 0.262}, {'id': 18, 'data': [136, 174, 25.75444900724711], 'fitness_val': 1.562}, {'id': 19, 'data': [169, 174, 25.36764337042531], 'fitness_val': 0.272}, {'id': 20, 'data': [194, 174, 14.53068195513866], 'fitness_val': 0.178}, {'id': 21, 'data': [199, 174, 1.578173374613003], 'fitness_val': 0.02}], [{'id': 22, 'data': [102, 181, 3.103490196078431], 'fitness_val': 0.048}, {'id': 23, 'data': [114, 181, 37.89713383311653], 'fitness_val': 0.455}, {'id': 24, 'data': [151, 181, 41.22557742534096], 'fitness_val': 0.459}, {'id': 25, 'data': [165, 181, 31.994281729273304], 'fitness_val': 0.483}, {'id': 26, 'data': [191, 181, 16.10551562608184], 'fitness_val': 0.308}, {'id': 27, 'data': [202, 181, 3.1699607843137305], 'fitness_val': 0.07}], [{'id': 28, 'data': [99, 189, 3.3224684295736644], 'fitness_val': 0.138}, {'id': 29, 'data': [133, 189, 13.331038045988182], 'fitness_val': 0.427}, {'id': 30, 'data': [157, 189, 23.540583140387078], 'fitness_val': 0.27}, {'id': 31, 'data': [169, 189, 27.898481299006175], 'fitness_val': 0.962}, {'id': 32, 'data': [190, 189, 22.58592795706785], 'fitness_val': 0.68}, {'id': 33, 'data': [199, 189, 11.223919409965816], 'fitness_val': 0.163}], [{'id': 34, 'data': [97, 199, 3.5001050420168065], 'fitness_val': 0.066}, {'id': 35, 'data': [118, 199, 39.8587439973304], 'fitness_val': 3.03}, {'id': 36, 'data': [154, 199, 41.40849849806095], 'fitness_val': 0.351}, {'id': 37, 'data': [154, 199, 39.94332671675898], 'fitness_val': 1.111}, {'id': 38, 'data': [184, 199, 18.964321520238247], 'fitness_val': 0.259}, {'id': 39, 'data': [197, 199, 13.155634453781511], 'fitness_val': 0.225}], [{'id': 40, 'data': [97, 209, 3.6409285714285713], 'fitness_val': 0.044}, {'id': 41, 'data': [126, 209, 31.76172605084069], 'fitness_val': 0.388}, {'id': 42, 'data': [140, 209, 51.344055826019606], 'fitness_val': 1.754}, {'id': 43, 'data': [149, 209, 45.445698272912054], 'fitness_val': 0.552}, {'id': 44, 'data': [151, 209, 34.22238290664472], 'fitness_val': 0.613}, {'id': 45, 'data': [197, 209, 13.959085434173668], 'fitness_val': 0.161}], [{'id': 46, 'data': [98, 219, 3.2516007130124773], 'fitness_val': 0.098}, {'id': 47, 'data': [127, 219, 31.877543895016423], 'fitness_val': 0.459}, {'id': 48, 'data': [129, 219, 44.657417317692314], 'fitness_val': 0.685}, {'id': 49, 'data': [153, 219, 40.68967253396035], 'fitness_val': 0.375}, {'id': 50, 'data': [159, 219, 29.400953082240722], 'fitness_val': 0.503}, {'id': 51, 'data': [198, 219, 11.102777183600718], 'fitness_val': 0.12}], [{'id': 52, 'data': [102, 228, 5.381042071197407], 'fitness_val': 0.105}, {'id': 53, 'data': [118, 228, 20.874376791803282], 'fitness_val': 0.358}, {'id': 54, 'data': [146, 228, 37.900742972751594], 'fitness_val': 0.909}, {'id': 55, 'data': [158, 228, 25.747719572997976], 'fitness_val': 0.424}, {'id': 56, 'data': [187, 228, 21.660516513736347], 'fitness_val': 0.422}, {'id': 57, 'data': [202, 228, 2.7783361888444698], 'fitness_val': 0.03}], [{'id': 58, 'data': [119, 235, 17.441044117647063], 'fitness_val': 0.36}, {'id': 59, 'data': [127, 235, 39.94103233681906], 'fitness_val': 0.505}, {'id': 60, 'data': [151, 235, 27.532191363402198], 'fitness_val': 0.538}, {'id': 61, 'data': [181, 235, 11.082319356099129], 'fitness_val': 0.142}, {'id': 62, 'data': [199, 235, 1.4457107843137258], 'fitness_val': 0.028}], [{'id': 63, 'data': [116, 241, 11.487032679738556], 'fitness_val': 0.257}, {'id': 64, 'data': [136, 241, 42.15775158674534], 'fitness_val': 0.578}, {'id': 65, 'data': [175, 241, 27.871000590362474], 'fitness_val': 0.31}, {'id': 66, 'data': [179, 241, 14.417926154302695], 'fitness_val': 2.326}, {'id': 67, 'data': [196, 241, 1.700483660130719], 'fitness_val': 0.029}], [{'id': 68, 'data': [112, 247, 2.6447049258727913], 'fitness_val': 0.08}, {'id': 69, 'data': [144, 247, 22.658385172846494], 'fitness_val': 0.704}, {'id': 70, 'data': [169, 247, 22.843896777215623], 'fitness_val': 0.45}, {'id': 71, 'data': [191, 247, 11.312002999909609], 'fitness_val': 0.157}, {'id': 72, 'data': [192, 247, 1.6802735533237685], 'fitness_val': 0.037}], [{'id': 73, 'data': [124, 258, 5.506801470588235], 'fitness_val': 0.227}, {'id': 74, 'data': [158, 258, 11.660860369979714], 'fitness_val': 0.18}, {'id': 75, 'data': [163, 258, 3.293649108640842], 'fitness_val': 0.052}, {'id': 76, 'data': [184, 258, 1.683703431372549], 'fitness_val': 0.031}], [{'id': 77, 'data': [139, 263, 15.178675381263618], 'fitness_val': 0.181}, {'id': 78, 'data': [146, 263, 5.585180398980595], 'fitness_val': 0.121}, {'id': 79, 'data': [179, 263, 1.6793420479302832], 'fitness_val': 0.018}], [{'id': 80, 'data': [133, 267, 4.675844049247608], 'fitness_val': 0.098}, {'id': 81, 'data': [143, 267, 2.9098048838148776], 'fitness_val': 0.04}, {'id': 82, 'data': [173, 267, 1.8026675786593707], 'fitness_val': 0.021}]], 'fitNum': 25.664, 'objEdges': [[0, 1], [1, 2], [2, 3], [3, 4], [3, 7], [4, 5], [4, 8], [5, 6], [5, 9], [6, 10], [7, 8], [7, 11], [7, 12], [8, 9], [8, 12], [8, 13], [9, 10], [9, 12], [9, 13], [10, 12], [10, 13], [11, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, 15], [14, 17], [15, 16], [15, 17], [16, 17], [16, 18], [16, 19], [16, 20], [16, 21], [17, 18], [17, 22], [17, 23], [18, 19], [18, 23], [18, 24], [19, 20], [19, 24], [19, 25], [19, 26], [20, 21], [20, 26], [21, 26], [21, 27], [22, 23], [22, 28], [23, 24], [23, 29], [24, 25], [24, 30], [25, 26], [25, 31], [26, 27], [26, 32], [27, 33], [28, 29], [28, 34], [29, 30], [29, 35], [30, 31], [30, 36], [31, 32], [31, 37], [32, 33], [32, 38], [33, 39], [34, 35], [34, 40], [35, 36], [35, 41], [36, 37], [36, 42], [37, 38], [37, 43], [38, 39], [38, 44], [39, 45], [40, 41], [40, 46], [41, 42], [41, 47], [42, 43], [42, 48], [43, 44], [43, 49], [44, 45], [44, 50], [45, 51], [46, 47], [46, 52], [47, 48], [47, 53], [48, 49], [48, 54], [49, 50], [49, 55], [50, 51], [50, 56], [51, 57], [52, 53], [52, 58], [53, 54], [53, 58], [53, 59], [54, 55], [54, 59], [55, 56], [55, 59], [55, 60], [55, 61], [55, 62], [56, 57], [56, 59], [56, 60], [56, 61], [56, 62], [57, 59], [57, 60], [57, 61], [57, 62], [58, 59], [58, 63], [59, 60], [59, 64], [60, 61], [60, 65], [61, 62], [61, 66], [62, 67], [63, 64], [63, 68], [64, 65], [64, 69], [65, 66], [65, 70], [66, 67], [66, 71], [67, 72], [68, 69], [68, 73], [69, 70], [69, 73], [69, 74], [70, 71], [70, 74], [71, 72], [71, 74], [71, 75], [71, 76], [72, 74], [72, 75], [72, 76], [73, 74], [73, 77], [74, 75], [74, 77], [74, 78], [75, 76], [75, 78], [75, 79], [76, 78], [76, 79], [77, 78], [77, 80], [78, 79], [78, 81], [79, 82]]}

print(len(p1), len(p2))

In [68]:
# 'objDict': [], write this
# 'fitNum': 0, compute 
# 'objEdges': [], compute 
import random 

g3 = p1['objDict']
# print(g3)

for i in range(len(p1['objDict']))[3:4:]:
    for j in range(len(p1['objDict'][i])):
        # print(p1['objDict'][i][j], p2['objDict'][i][j])
        
        p = random.randint(50, 100)
        # print(p)
        
        if (p1['objDict'][i][j]['fitness_val'] > p2['objDict'][i][j]['fitness_val']):
            fittest = p1['objDict'][i][j]
            lessFittest = p2['objDict'][i][j]
        else:
            fittest = p2['objDict'][i][j]
            lessFittest = p1['objDict'][i][j]
            
        # print("Fittest: ", fittest)
        # print("Less fittest: ", lessFittest)

        x = abs(p1['objDict'][i][j]['data'][0] - p2['objDict'][i][j]['data'][0])
        # print('X: ', x)
        
        distX = round(x * p/100)
        # print('DistX: ', distX)

        #check x vals
        if(fittest['data'][0] > lessFittest['data'][0]):
            fittestXLarger = True
        else:
            fittestXLarger = False
        
        # print(fittestXLarger)

        #toward p with lower f.val
        if (fittestXLarger):
            g3[i][j]['data'][0] = fittest['data'][0] - distX 
        else:
            g3[i][j]['data'][0] = fittest['data'][0] + distX

        z = abs(p1['objDict'][i][j]['data'][2] - p2['objDict'][i][j]['data'][2])
        # print('Z: ', z)
        
        distZ = round(z * p/100)
        # print('DistZ: ', distZ)

        #check z vals
        if(fittest['data'][2] > lessFittest['data'][2]):
            fittestZLarger = True
        else:
            fittestZLarger = False
        
        # print(fittestZLarger)

        #toward p with higher f.val
        if (fittestZLarger):
            g3[i][j]['data'][2] = lessFittest['data'][2] + distZ 
        else:
            g3[i][j]['data'][2] = lessFittest['data'][2] - distZ

        g3[i][j]['fitness_val'] = 0
            
        print('G3: ', g3[i][j])

# print(g3)


G3:  {'id': 3, 'data': [127, 156, 2.034078431372549], 'fitness_val': 0}
G3:  {'id': 4, 'data': [132, 156, 14.675402423839277], 'fitness_val': 0}
G3:  {'id': 5, 'data': [142, 156, 12.396760736653265], 'fitness_val': 0}
G3:  {'id': 6, 'data': [169, 156, 1.6688057040998214], 'fitness_val': 0}


In [62]:
#mutation
for i in range(len(g3))[3:4:]:
    for j in range(len(g3[i])):
        if(len(g3[i]) > 2):
            if (j != 0 and j != len(g3[i])-1):
                print('G3: ', g3[i][j])
                r = random.randint(0, 10)
                print(r)
                
                g3[i][j]['data'][2] += r
                
                print('G3M: ', g3[i][j])

G3:  {'id': 4, 'data': [132, 156, 14.675402423839277], 'fitness_val': 0}
8
G3M:  {'id': 4, 'data': [132, 156, 22.675402423839277], 'fitness_val': 0}
G3:  {'id': 5, 'data': [142, 156, 12.396760736653265], 'fitness_val': 0}
5
G3M:  {'id': 5, 'data': [142, 156, 17.396760736653263], 'fitness_val': 0}


In [ ]:
# for generation in range(n) do evolution 
# store children in new_gen[] (will be of len n)
# creatures = new_gen[] ???? pop.creatures gets rewritten

#Elitism


In [11]:
verts = data['objVerts']
edges = data['objEdges']

In [ ]:
arr6 = edges
        
#add to edges array in the form of [[0, 1], [1, 2]]
faces = []

for i in range(len(arr6)):
    for j in range(len(arr6[i])):
        if(len(arr5[i]) < 2):  
            edges.append([arr5[i][j]['id'], arr5[i+1][j]['id']])
        else:
            #when next array is larger or smaller than the current array
            if (i != len(arr5)-1):
                if (len(arr5[i]) != len(arr5[i+1])):
                    #when j is not the last index
                    if (j != len(arr5[i])-1):
                        edges.append([arr5[i][j]['id'], arr5[i][j+1]['id']])
        
                        if (i != len(arr5)-1):  
                            tempDist = []
        
                            if(j == 0):
                                count = 0
                            else:
                                count = ind
        
                            for c in range(len(arr5[i+1]))[count::]:
                                x = pow(abs(arr5[i][j]['data'][0] - arr5[i+1][c]['data'][0]), 2)
                                y = pow(abs(arr5[i][j]['data'][1] - arr5[i+1][c]['data'][1]), 2)
                                dist = round(math.sqrt(x + y), 2)
        
                                edges.append([arr5[i][j]['id'], arr5[i+1][c]['id']])
                                tempDist.append(dist)
                                # print(arr5[i][j]['id'], arr5[i+1][c]['id'], tempDist)
                                
                                if (len(tempDist) > 0 and dist > tempDist[0]):
                                    edges.pop()
                                    ind = c-1
                                    break
                    # when j is the last index
                    else:
                        if(i != len(arr5)-1):
                            for c in range(len(arr5[i+1]))[ind::]:
                              edges.append([arr5[i][j]['id'], arr5[i+1][c]['id']])
                #when arrays are equal
                else:
                    if (j != len(arr5[i])-1):
                        edges.append([arr5[i][j]['id'], arr5[i][j+1]['id']])
                        if (i != len(arr5)-1):
                            edges.append([arr5[i][j]['id'], arr5[i+1][j]['id']])
                    else:
                        if(i != len(arr5)-1):
                            edges.append([arr5[i][j]['id'], arr5[i+1][len(arr5[i+1])-1]['id']])